## 실습: 은행

### 단계 1:컨트랙 개발

In [2]:
%%writefile src/BankV3.sol
pragma solidity 0.5.1;
contract BankV3 {
    address owner;
    uint balance;//balance: 여러사람의 잔고 기록할 수 있어야한다. 컨트랙의 잔고 추적 기능
    uint256 timeToDeposit;//시간
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() public {
        owner = msg.sender;//owner초기화
        balance = 0;
    }

    function() external {
        emit PrintLog("Fallback called");
    }
    function forwardTo(address payable _receiver) public payable onlyOwner {//owner일 경우만 실행
        //require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);//계좌 이체시 sent event 발생
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);//2개 이상 return시 괄호로 묶어서
    }
    function deposit(uint amount) public payable onlyAfter {//매개변수 없을때: value에 넣는다. 여기서는 amount에 넣음
        timeToDeposit = now + 10 seconds;//10초이후
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {//잔고를 전체 인출, owner만, 최소잔고 있을 때
        balance -= address(this).balance;
        //require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }
    modifier onlyOwner {
        require(msg.sender == owner);//조건 만족할 때만 아래 실행
        _;
    }
    modifier onlyAfter {
        require(now >= timeToDeposit);//조건 만족할 때만 아래 실행
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);//조건 만족할 때만 아래 실행
        _;
    }
}

Overwriting src/BankV3.sol


### 단계 2: 컴파일

In [3]:
!solc --gas --abi --bin src/BankV3.sol


======= src/BankV3.sol:BankV3 =======
Gas estimation:
construction:
   25549 + 225600 = 251149
external:
   fallback:	1795
   deposit(uint256):	40764
   forwardTo(address):	infinite
   getBalance():	835
   withdrawAll():	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610468806100686000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe0146100d357806327d8ad8814610105578063853828b614610149578063b6b55f2514610160575b34801561006857600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100df57600080fd5b506100e861018e565b604051808381526020018281526020019250505060405180910390f35b6101476004803603602081101561

### 단계 3: 컨트랙 배포

In [14]:
%%writefile src/BankV3Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin = "0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610468806100686000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe0146100d357806327d8ad8814610105578063853828b614610149578063b6b55f2514610160575b34801561006857600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100df57600080fd5b506100e861018e565b604051808381526020018281526020019250505060405180910390f35b6101476004803603602081101561011b57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506101b4565b005b34801561015557600080fd5b5061015e6102f8565b005b61018c6004803603602081101561017657600080fd5b8101908080359060200190929190505050610400565b005b6000806001543073ffffffffffffffffffffffffffffffffffffffff1631915091509091565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561020f57600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f19350505050158015610255573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561035357600080fd5b60653073ffffffffffffffffffffffffffffffffffffffff163111151561037957600080fd5b3073ffffffffffffffffffffffffffffffffffffffff16316001600082825403925050819055503373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f193505050501580156103fd573d6000803e3d6000fd5b50565b600254421015151561041157600080fd5b600a4201600281905550803414151561042957600080fd5b806001600082825401925050819055505056fea165627a7a7230582072fac3f93f4d45f9c84a01258965d0a0a47e559a0307ef335fface1c7031cb1f0029";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();//값 반환 기다림
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/BankV3Deploy.js


In [15]:
!node src/BankV3Deploy.js

Deploying the contract from 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
hash: 0x815f3e848175f5824c4b424f4b3c88da801fe1a0593c6146f17729d1738d9c83
---> The contract deployed to: 0xa39d557964b8D76cE1b602f931e674Ad4fDa35E7


### Available Accounts

(0) 0x008D3582CBB59Ba9B39be6A03C43528F90416f20 (100 ETH)
(1) 0x6319F203e91EF3D904E0Ac3797b2e3a65342aED2 (100 ETH)
(2) 0x194C327B379cf529294387123C0583806B58d7C2 (100 ETH)
(3) 0xe29dB2951413695D52e11A40E3132e26978021b0 (100 ETH)
(4) 0xaA8823Fd675C97E8A70455f0801C5e8edbAE9F56 (100 ETH)
(5) 0x5Aaa780E547482d95f4f1f841C83A04F56c10aEC (100 ETH)
(6) 0xfd4F53Ca551D18289168C01442eBEdd84bc2AEE2 (100 ETH)
(7) 0xB6f629f04bc45DD198D650604eC630bB9FE0A601 (100 ETH)
(8) 0x37f86b6FFe4b7B477E5172EeA7BA8Fd93dF82160 (100 ETH)
(9) 0xC69646f72C354685cFD1227eB921853a5161481e (100 ETH)

##### 첫 번째 주소 출력됨

### 단계 4. 사용

프로그램에 설정한 제약조건을 지켜서 실행헤야 한다.

- 10초 이내 저축
- 잔고 101보다 적은데 출금

In [16]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var bank = new web3.eth.Contract(_abiArray, "0xa39d557964b8D76cE1b602f931e674Ad4fDa35E7");

//이벤트 생성할 때 만들어주는 코드
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.deposit(111).send({from: accounts[0], value:111}); //value: 내 잔고에서 빠져서 contract로 가는.. 앞의 (111)과 같아야함
    bank.methods.getBalance().call().then(console.log);//await로 했기 때문에 바로 반영됨
    await bank.methods.withdrawAll().send({from: accounts[0]});    //greater than 101
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));//숫자끼리: 연산
    
}
doIt()

Writing src/BankV3Use.js


In [17]:
!node src/BankV3Use.js

Account: 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
Balance before: 99980847700000000000
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Result { '0': '0', '1': '0' }
Balance after: 99979132520000000000
Balance diff: 1715180000002048


---

## 예제: 간단한 이벤트

### 단계 1: 컨트랙 개발

In [1]:
%%writefile src/EventTest.sol
pragma solidity 0.5.1;
contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting src/EventTest.sol


### 단계 2: 컴파일

In [2]:
!solc --abi --bin --gas src/EventTest.sol


======= src/EventTest.sol:EventTest =======
Gas estimation:
construction:
   99 + 46400 = 46499
external:
   myFunction():	1753
Binary: 
6080604052348015600f57600080fd5b5060e88061001e6000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063c3780a3a14603e575b600080fd5b348015604957600080fd5b5060506052565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea165627a7a7230582051a28a153e2662cebe01363714a90ded6c0c72a97c60ce8b074be20d4d43a30f0029
Contract JSON ABI 
[{"constant":false,"inputs":[],"name":"myFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"my","type":"string"}],"name":"MyLog","type":"event"}]


### 단계 3: 컨트랙 배포

In [3]:
%%writefile src/EventTestDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray=[{"constant":false,"inputs":[],"name":"myFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"my","type":"string"}],"name":"MyLog","type":"event"}];
var _bin="0x"+"6080604052348015600f57600080fd5b5060e88061001e6000396000f3fe6080604052600436106039576000357c010000000000000000000000000000000000000000000000000000000090048063c3780a3a14603e575b600080fd5b348015604957600080fd5b5060506052565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea165627a7a7230582051a28a153e2662cebe01363714a90ded6c0c72a97c60ce8b074be20d4d43a30f0029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();//await로 계정을 구함
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {//gas는 임의의 숫자
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/EventTestDeploy.js


사설망: 암호풀고 마이닝

In [4]:
!node src/EventTestDeploy.js

Deploying the contract from 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
hash: 0x5f9e8b1d947ba8780c3220951350ccbe84ff2dfc31f2b78fb0c48214f3f9b10b
---> The contract deployed to: 0x2686F350Ee220C5Bce19aDB0B02A55E47633f53d


In [5]:
##hash를 넣는다
!geth --exec "eth.getTransactionReceipt('0x5f9e8b1d947ba8780c3220951350ccbe84ff2dfc31f2b78fb0c48214f3f9b10b');" attach http://localhost:8345

{
  blockHash: "0x3a3dbaf17b541993858d1adc8a75bbbf561fed254dbd5cf9cf6ffcb70f12e615",
  blockNumber: 10,
  contractAddress: "0x2686f350ee220c5bce19adb0b02a55e47633f53d",
  cumulativeGasUsed: 103007,
  from: "0x008d3582cbb59ba9b39be6a03c43528f90416f20",
  gasUsed: 103007,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x5f9e8b1d947ba8780c3220951350ccbe84ff2dfc31f2b78fb0c48214f3f9b10b",
  transactionIndex: 0
}


construction에 나온 gas, binary에서 0이아닌 것들의 gas비 계산


### 단계 4: 사용 

In [8]:
%%writefile src/EventTestHttpNoEventFiredUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//http provider로하면 안된다

var _abiArray=[{"constant":false,"inputs":[],"name":"myFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"my","type":"string"}],"name":"MyLog","type":"event"}];
var _test = new web3.eth.Contract(_abiArray, '0x2686F350Ee220C5Bce19aDB0B02A55E47633f53d');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called "+JSON.stringify(value)+
        //                               '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called "+JSON.stringify(value)+
        '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));
    
    
    console.log("print only string: ",value.events.MyLog.returnValues[0]);
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/EventTestHttpNoEventFiredUse.js


In [9]:
!node src/EventTestHttpNoEventFiredUse.js

Account: 0x008D3582CBB59Ba9B39be6A03C43528F90416f20
Balance before: 99978492322000000000
---> myFunction called {"transactionHash":"0xb9719f45f839d2f055eadf9b71315aed7f1e35bbee9edda8b37cf6c28f785ac8","transactionIndex":0,"blockHash":"0x7cd63cd2a0e6532ca2e7e456811112f38478a4d273de82389deef4cefdef552f","blockNumber":20,"from":"0x008d3582cbb59ba9b39be6a03c43528f90416f20","to":"0x2686f350ee220c5bce19adb0b02a55e47633f53d","gasUsed":22817,"cumulativeGasUsed":22817,"contractAddress":null,"status":true,"logsBloom":"0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000010000000200000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000

In [1]:
%%writefile src/EventTestWsUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var _abiArray=[{"constant":false,"inputs":[],"name":"myFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"my","type":"string"}],"name":"MyLog","type":"event"}];
var _test = new web3.eth.Contract(_abiArray, '0x2686F350Ee220C5Bce19aDB0B02A55E47633f53d');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called "+JSON.stringify(value)+
        //                               '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called "+JSON.stringify(value)+
        '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/EventTestWsUse.js


In [ ]:
!node src/EventTestWsUse.js

웹소켓->무한루프